In [1]:
pip install -q -r ./requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.1/366.1 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

# Load API Keys
load_dotenv(find_dotenv(), override=True)

open_ai_key =os.environ.get('OPENAI_API_KEY')
google_gemini_key = os.environ.get("GOOGLE_GEMINI_API_KEY")
google_maps_key = os.environ.get("GOOGLE_MAPS_API_KEY")

In [3]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# Define Validation class for pydantic Field.
# This allows you to specify validation rules for fields and set default values
class Validation(BaseModel):
  plan_is_valid: str = Field(
      description="This field is 'True' if the plan is feasible, 'False' otherwise"
  )
  updated_request: str = Field(description="Your update to the plan")

# Define ValidationTemplate
class ValidationTemplate(object):
  def __init__(self):
    # System template gives context and sets the stage for LLM
    self.system_template = """
      You are a travel planner agent who helps users make detailed and accurate travel plans.

      The user's request will be denoted by four hashtags. Determine if the user's
      request is reasonable and achievable within the constraints they set.

      A valid request should contain the following:
      - A start and end location
      - A trip duration that is reasonable given the start and end location
      - Some other details, like the user's interests and/or preferred mode of transport

      Any request that is not viable or contains potentially dangerous or harmful activities is not valid,
      regardless of what other details are provided.

      If the request is not valid, set
      plan_is_valid = "0" and use your travel knowledge to update the request to make it valid,
      keeping the request within 100 words.

      Otherwise, if the request seems reasonable, set plan_is_valid = "1" and don't revise the request.

      {format_instructions}
    """

    # Human template is inputted as a query variable
    self.human_template = """
      ####{query}####
    """

    self.parser = PydanticOutputParser(pydantic_object=Validation)

    self.system_message_prompt = SystemMessagePromptTemplate.from_template(
        self.system_template,
        partial_variables={
            "format_instructions": self.parser.get_format_instructions()
        },
    )
    self.human_message_prompt = HumanMessagePromptTemplate.from_template(
        self.human_template, input_variables=["query"]
    )

    self.chat_prompt = ChatPromptTemplate.from_messages(
        [self.system_message_prompt, self.human_message_prompt]
    )

In [7]:
# Define ItineraryTemplate
class ItineraryTemplate(object):
    def __init__(self):
        self.system_template = """
      You are a travel planner agent who helps users make detailed and accurate travel plans.

      The user's request will be denoted by four hashtags. Convert the
      user's request into a detailed itinerary describing the places
      they should visit and the things they should do.

      For each location, provide the specific address.

      Remember to take the user's preferences and timeframe into account,
      and give them an itinerary that would be fun and doable given their constraints.

      Return the itinerary as a bulleted list with clear start and end locations.
      Be sure to mention the type of transit for the trip.
      If specific start and end locations are not given, choose ones that you think are suitable and give specific addresses.
      Your output must be the list and nothing else.
    """

        self.human_template = """
      ####{query}####
    """

        self.system_message_prompt = SystemMessagePromptTemplate.from_template(
            self.system_template,
        )
        self.human_message_prompt = HumanMessagePromptTemplate.from_template(
            self.human_template, input_variables=["query"]
        )

        self.chat_prompt = ChatPromptTemplate.from_messages(
            [self.system_message_prompt, self.human_message_prompt]
        )

In [12]:
# Define Trip class for pydantic Field.
class Trip(BaseModel):
    start: str = Field(description="start location of trip")
    end: str = Field(description="end location of trip")
    stops: list[str] = Field(description="list of stops")
    transit: str = Field(description="mode of transportation")

# Define MappingTemplate
class MappingTemplate(object):
    def __init__(self):
        self.system_template = """
      You are an agent who converts detailed travel plans into a simple list of locations.

      The itinerary will be denoted by four hashtags. Convert it into
      list of places that they should visit. Try to include the specific address of each location.

      Your output should always contain the start and end point of the trip, and may also include a list
      of stops. It should also include a mode of transit. The number of stops cannot exceed 20.
      If you can't infer the mode of transit, make a best guess given the trip location.

      For example:

      ####
      Itinerary for a 2-day driving trip within London:
      - Day 1:
        - Start at Buckingham Palace (The Mall, London SW1A 1AA)
        - Visit the Tower of London (Tower Hill, London EC3N 4AB)
        - Explore the British Museum (Great Russell St, Bloomsbury, London WC1B 3DG)
        - Enjoy shopping at Oxford Street (Oxford St, London W1C 1JN)
        - End the day at Covent Garden (Covent Garden, London WC2E 8RF)
      - Day 2:
        - Start at Westminster Abbey (20 Deans Yd, Westminster, London SW1P 3PA)
        - Visit the Churchill War Rooms (Clive Steps, King Charles St, London SW1A 2AQ)
        - Explore the Natural History Museum (Cromwell Rd, Kensington, London SW7 5BD)
        - End the trip at the Tower Bridge (Tower Bridge Rd, London SE1 2UP)
      #####

      Output:
      Start: Buckingham Palace, The Mall, London SW1A 1AA
      End: Tower Bridge, Tower Bridge Rd, London SE1 2UP
      Waypoints: ["Tower of London, Tower Hill, London EC3N 4AB", "British Museum, Great Russell St, Bloomsbury, London WC1B 3DG", "Oxford St, London W1C 1JN", "Covent Garden, London WC2E 8RF","Westminster, London SW1A 0AA", "St. James's Park, London", "Natural History Museum, Cromwell Rd, Kensington, London SW7 5BD"]
      Transit: driving

      Transit can be only one of the following options: "walking", "driving", "train", "bus", "subway", or "flight".

      {format_instructions}
    """

        self.human_template = """
      ####{agent_suggestion}####
    """

        self.parser = PydanticOutputParser(pydantic_object=Trip)

        self.system_message_prompt = SystemMessagePromptTemplate.from_template(
            self.system_template,
            partial_variables={
                "format_instructions": self.parser.get_format_instructions()
            },
        )
        self.human_message_prompt = HumanMessagePromptTemplate.from_template(
            self.human_template, input_variables=["agent_suggestion"]
        )

        self.chat_prompt = ChatPromptTemplate.from_messages(
            [self.system_message_prompt, self.human_message_prompt]
        )

In [9]:
import openai
import logging
import google.generativeai as genai
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SequentialChain

logging.basicConfig(level=logging.INFO)

# Define Agent class to 1) validate, 2) provide itinerary, 3) map itinerary addresses
class Agent(object):
    def __init__(
        self,
        open_ai_api_key,
        model="gpt-3.5-turbo",
        temperature=0,
        verbose=True,
    ):
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)
        self._openai_key = open_ai_api_key

        self.chat_model = ChatOpenAI(model=model, temperature=temperature, openai_api_key=self._openai_key)
        self.validation_prompt = ValidationTemplate()
        self.itinerary_prompt = ItineraryTemplate()
        self.mapping_prompt = MappingTemplate()
        self.validation_chain = self._set_up_validation_chain(verbose)
        self.agent_chain = self._set_up_agent_chain(verbose)

    def _set_up_validation_chain(self, verbose=True):
        validation_agent = LLMChain(
            llm=self.chat_model,
            prompt=self.validation_prompt.chat_prompt,
            output_parser=self.validation_prompt.parser,
            output_key="validation_output",
            verbose=verbose,
        )

        overall_chain = SequentialChain(
            chains=[validation_agent],
            input_variables=["query", "format_instructions"],
            output_variables=["validation_output"],
            verbose=verbose,
        )

        return overall_chain

    def _set_up_agent_chain(self, debug=True):
      travel_agent = LLMChain(
              llm=self.chat_model,
              prompt=self.itinerary_prompt.chat_prompt,
              verbose=debug,
              output_key="agent_suggestion",
      )

      parser = LLMChain(
              llm=self.chat_model,
              prompt=self.mapping_prompt.chat_prompt,
              output_parser=self.mapping_prompt.parser,
              verbose=debug,
              output_key="mapping_list",
      )

      overall_chain = SequentialChain(
              chains=[travel_agent, parser],
              input_variables=["query", "format_instructions"],
              output_variables=["agent_suggestion", "mapping_list"],
              verbose=debug,
      )

      return overall_chain

    def validate_travel(self, query):
        self.logger.info("Validating query with {} model".format(
                self.chat_model.model_name)
        )
        validation_result = self.validation_chain(
            {
                "query": query,
                "format_instructions": self.validation_prompt.parser.get_format_instructions(),
            }
        )

        validation_test = validation_result["validation_output"].dict()

        return validation_test

In [10]:
travel_agent = Agent(open_ai_api_key=open_ai_key, verbose=True)

query_valid = """
  I want to do a 7 day roadtrip from Berlin to Paris.
  I want to visit the famous attractions along the route.
  """
query_invalid = """
  I want to bike from Seoul, South Korea to Tokyo, Japan.
  After I arrive in Tokyo, I want to go to the moon.
    """

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [10]:
travel_agent.validate_travel(query_valid)

INFO:__main__:Validating query with gpt-3.5-turbo model




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
      You are a travel planner agent who helps users make detailed and accurate travel plans.

      The user's request will be denoted by four hashtags. Determine if the user's
      request is reasonable and achievable within the constraints they set.

      A valid request should contain the following:
      - A start and end location
      - A trip duration that is reasonable given the start and end location
      - Some other details, like the user's interests and/or preferred mode of transport

      Any request that is not viable or contains potentially dangerous or harmful activities is not valid, 
      regardless of what other details are provided.

      If the request is not valid, set
      plan_is_valid = "0" and use your travel knowledge to update the request to make it valid,
      keeping the request within 100 words.

      Otherwise, if the request seems rea

{'plan_is_valid': '1', 'updated_request': 'No update needed'}

In [63]:
travel_agent.validate_travel(query_invalid)

INFO:__main__:Validating query with gpt-3.5-turbo model




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
      You are a travel planner agent who helps users make detailed and accurate travel plans.

      The user's request will be denoted by four hashtags. Determine if the user's
      request is reasonable and achievable within the constraints they set.

      A valid request should contain the following:
      - A start and end location
      - A trip duration that is reasonable given the start and end location
      - Some other details, like the user's interests and/or preferred mode of transport

      Any request that contains potentially harmful activities is not valid, regardless of what
      other details are provided.

      If the request is not valid, set
      plan_is_valid = "0" and use your travel knowledge to update the request to make it valid,
      keeping your revised request within 100 words.

      If the request seems reasonable, then set plan_is_valid =

{'plan_is_valid': '0',
 'updated_request': "The request is not valid as biking from Seoul, South Korea to Tokyo, Japan is not feasible due to the distance and the fact that crossing the ocean by bike is not possible. Additionally, traveling to the moon is currently not achievable for civilians. Please provide a more realistic travel plan within Earth's boundaries."}

In [13]:
agent_chain = travel_agent._set_up_agent_chain()
mapping_prompt = MappingTemplate()

agent_result = agent_chain(
                {
                    "query": query_valid,
                    "format_instructions": mapping_prompt.parser.get_format_instructions(),
                }
            )

trip_suggestion = agent_result["agent_suggestion"]
stops_dict = agent_result["mapping_list"].dict()



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: 
      You are a travel planner agent who helps users make detailed and accurate travel plans.

      The user's request will be denoted by four hashtags. Convert the
      user's request into a detailed itinerary describing the places
      they should visit and the things they should do.

      For each location, provide the specific address.

      Remember to take the user's preferences and timeframe into account,
      and give them an itinerary that would be fun and doable given their constraints.

      Return the itinerary as a bulleted list with clear start and end locations.
      Be sure to mention the type of transit for the trip.
      If specific start and end locations are not given, choose ones that you think are suitable and give specific addresses.
      Your output must be the list and nothing else.
    
Human: 
      ####
  I want to do a 7 day roadtrip from